In [1]:
import cv2
import numpy as np
import face_recognition
from PIL import Image
import glob
import re
import ntpath
import os
import dlib
import time
from imutils.face_utils import FaceAligner
import sqlite3

In [ ]:
image_list_file = []

In [ ]:
for filename in glob.glob('/home/damvantai/Documents/face_recognition_demo/data/pictures_of_people_i_know/*'):
    image = Image.open(filename)
    image_list_file.append(filename)

In [ ]:
len(image_list_file)

In [ ]:
name_image = ntpath.split(ntpath.split(image_list_file[1])[0])[1]

In [ ]:
ntpath.split(image_list_file[12])

In [ ]:
image_names = []
image_names = [ntpath.split(name)[1].split('.')[0] for name in image_list_file]

In [ ]:
known_face_encodings = []
known_face_names = []

In [ ]:
start = time.time()
for i, filename in enumerate(image_list_file):
    image = face_recognition.load_image_file(filename)
    try:
        temp_encoding = face_recognition.face_encodings(image)[0]
        known_face_encodings.append(temp_encoding)
        known_face_names.append(image_names[i])
    except IndexError:
        continue
end = time.time()
print("Time for read file and encoding: ", (end - start))

In [ ]:
# len(known_face_encodings)

In [ ]:
print(type(known_face_encodings))
print(type(known_face_encodings[0]))
print(type(known_face_encodings[0][0]))
print(known_face_names)

In [ ]:
print(type(known_face_encodings))
print(type(known_face_encodings[0]))
# print(known_face_encodings.shape)
print(known_face_encodings[0].shape)
print(type(known_face_encodings[0][0]))

In [ ]:
known_face_encodings[0]

In [ ]:
known_face_encodings

In [ ]:
known_face_encodings = np.load("../data/numpy/known_face_encoding.npy")
known_face_names = np.load("../data/numpy/known_face_names.npy")

# known_face_encodings = known_face_encodings.tolist()

In [ ]:
print(type(known_face_encodings))

In [ ]:
print(type(known_face_encodings[0]))
print(type(known_face_encodings[0][0]))
print(type(known_face_encodings[0][0][0]))
print(known_face_encodings[0].reshape(128))

In [ ]:
known_face_encodings.shape

In [ ]:
known_face_encodings = known_face_encodings.reshape(42, 128)

In [ ]:
print(known_face_encodings.shape)
print(known_face_encodings[0].shape)
print(known_face_encodings[0][0])
print(type(known_face_encodings))
print(type(known_face_encodings[0]))
print(type(known_face_encodings[0][0]))

In [ ]:
print(len(known_face_encodings))

In [ ]:
known_face_encodings_list = []
for i in range(len(known_face_encodings)):
    known_face_encodings_list.append(known_face_encodings[i])

In [ ]:
known_face_encodings_list

In [ ]:
print(type(known_face_encodings))
# known_face_encodings = known_face_encodings

In [ ]:
known_face_encodings[1]

In [ ]:
a = known_face_encodings.tolist()

In [ ]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
name = ""
frame_number = 0
known_face_names

In [1]:
import cv2
import numpy as np
import face_recognition
from PIL import Image
import glob
import re
import ntpath
import os
import dlib
import time
from imutils.face_utils import FaceAligner
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import dlib
import imutils
from imutils import face_utils

In [2]:
def face_center(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    rects = detector(image_gray, 1)
    for (i, rect) in enumerate(rects):
        shape = predictor(image_gray, rect)
        shape = face_utils.shape_to_np(shape)
        center_face = shape[37][1] - shape[46][1]
        if center_face < 10 & center_face > -10:
            return True
        else:
            return False

In [3]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
name = ""
frame_number = 0

known_face_encodings_array = np.load("../data/numpy/known_face_encoding.npy")
known_face_names = np.load("../data/numpy/known_face_names.npy")

len_known_face_encoding = known_face_encodings_array.shape[0]
known_face_encodings_array = known_face_encodings_array.reshape(len_known_face_encoding, 128)

known_face_encodings = []
for i in range(len(known_face_encodings_array)):
    known_face_encodings.append(known_face_encodings_array[i])

unknown_face_encodings = []
    
# Capture from camera of own computer
video_capture = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    # Resize frame of video to 1/4 size for faster face recognition processing
    # Frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    frame_number += 1
    
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = frame[:, :, ::-1]
        
    # Use computer configuration low
    if (frame_number % 10 == 0):
        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            # print(face_locations)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:

                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.4)

#                 print(face_recognition.face_distance(known_face_encodings, face_encoding))
                point = np.min(face_recognition.face_distance(known_face_encodings, face_encoding))
                print(point)
                
                # Create name for unknow people
                if point > 0.4:
                    if os.path.isfile("../data/unknow_people/unknown_0.jpg"):
                        a = os.listdir("../data/unknow_people/")
                        a.sort()
                        index = int(a[-1].split('.')[0].split('_')[1])
                        index += 1
                        path = "../data/unknow_people/unknown_" + str(index) + ".jpg"
    #                     cv2.imwrite(path, )
                        name = "unknown_" + str(index)
                    else:
                        name = "unknown_0"
                
                
                
                # If a match was found in known_face_encodings, just use the first one.
                if True in matches:
                    print(matches)
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                face_names.append(name)
                print(name)
    # process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        first_name = name.split("_")[0]
        if first_name == "unknown":
            face_unknow = frame[top:bottom, left:right]
            if face_center(face_unknow) == True:
                path = "../data/unknow_people/" + name + ".jpg"
                cv2.imwrite(path, face_unknow)
#                 face_location_unknow = face_recognition.face_locations(face_unknow)
                face_encoding_unknow = face_recognition.face_encodings(face_unknow, (top, right, bottom, left))
                unknown_face_encodings.append(face_encoding_unknow)
                print(len(unknown_face_encodings))
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        
        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

0.377320291275
[False, True, False, False]
Dam Van Tai
0.519653968268
unknown_0


TypeError: 'int' object is not subscriptable

In [3]:
import os
import ntpath

In [4]:
import cv2

In [3]:
abc = cv2.imread("../data/unknow_people/unknown_0.jpg", 1)

In [4]:
cv2.imshow('image', abc)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
path = '../data/unknow_people/unknown_' + str(4) + ".jpg"

In [15]:
cv2.imwrite(path, abc)

True

In [6]:
a = "unknown_1"

In [5]:
a = os.listdir("../data/unknow_people/")

In [8]:
a.sort()

In [9]:
a

['unknown_0.jpg', 'unknown_1.jpg', 'unknown_2.jpg', 'unknown_4.jpg']

In [5]:
if os.path.isfile("../data/unknow_people/unknown_0.jpg"):
    a = os.listdir("../data/unknow_people/")
    print(a[-1])
else:
    print("sai")

unknown_1.jpg


In [8]:
a.split("_")[0]

'unknown'

In [ ]:
int(a[-1].split('.')[0].split('_')[1])

In [ ]:
cv.

In [ ]:
# known_face_encodings = np.load("../data/numpy/known_face_encoding.npy")
# known_face_names = np.load("../data/numpy/known_face_names.npy")

# # for face detection
# detector = dlib.get_frontal_face_detector()
# predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
# fa_image = FaceAligner(predictor, desiredFaceWidth=160)

# # Capture from camera of own computer
# video_capture = cv2.VideoCapture(0)

# while True:
#     # Grab a single frame of video
#     ret, frame = video_capture.read()

#     # Resize frame of video to 1/4 size for faster face recognition processing
#     # Frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
#     frame_number += 1
# #     
#     # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
#     rgb_small_frame = frame[:, :, ::-1]
        
#     # Use computer configuration low
#     if (frame_number % 10 == 0):
#         face_encodings = []
#         # Only process every other frame of video to save time
#         if process_this_frame:
#             # Find all the faces and face encodings in the current frame of video
# #             face_locations = face_recognition.face_locations(rgb_small_frame)
# #             print(face_locations)
# #             face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            
#             # Align face
#             image_gray = cv2.cvtColor(rgb_small_frame, cv2.COLOR_RGB2GRAY)
#             detected_gray = detector(image_gray, 2)
#             print(detected_gray)
#             for i, d in enumerate(detected_gray):
#                 image_align = fa_image.align(rgb_small_frame, image_gray, detected_gray[i])
#                 face_encodings.append(face_recognition.face_encodings(image_align))
            
            
#             face_names = []
#             face_encodings = np.asarray(face_encodings)
#             for face_encoding in face_encodings:
#                 if len(face_encoding) > 0:
#                     face_encoding = np.asarray(face_encodings).reshape(128, )

#                     # See if the face is a match for the known face(s)
#                     matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.4)

#         #             print(face_recognition.face_distance(known_face_encodings, face_encoding))
#                     print(np.min(face_recognition.face_distance(known_face_encodings, face_encoding)))
#                     name = "Unknown"

#                     # If a match was found in known_face_encodings, just use the first one.
#                     if True in matches:
#                         first_match_index = matches.index(True)
#                         name = known_face_names[first_match_index]

#                     print(name)

#                     face_names.append(name)
#         for d, name in zip(detected_gray, face_names):
#             # Draw a label with a name below the face
#             cv2.rectangle(frame, (d.left(), d.bottom() - 35), (d.right(), d.bottom()), (0, 0, 255), cv2.FILLED)
#             font = cv2.FONT_HERSHEY_DUPLEX
#             cv2.putText(frame, name, (d.left() + 6, d.bottom() - 6), font, 1.0, (255, 255, 255), 1)
            
    
#     # Display the results
#     for (top, right, bottom, left), name in zip(face_locations, face_names):
#         # Scale back up face locations since the frame we detected in was scaled to 1/4 size
# #         top *= 4
# #         right *= 4
# #         bottom *= 4
# #         left *= 4

#         # Draw a box around the face
#         cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        
#         # Draw a label with a name below the face
#         cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
#         font = cv2.FONT_HERSHEY_DUPLEX
#         cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

#     # Display the resulting image
#     cv2.imshow('Video', frame)

#     # Hit 'q' on the keyboard to quit!
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break